<a href="https://colab.research.google.com/github/PranavkrishnaVadhyar/Knowledge-Navigator/blob/main/Knowledge_navigator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install langchain_experimental

In [6]:
!pip install --upgrade --quiet  langchain sentence_transformers
!pip install langchain
!pip install langchain_community

In [7]:
with open("/content/drive/MyDrive/SparkVenture First aid pro action plan.txt","r",encoding='utf-8') as f:
    data = f.read()



In [8]:
data

'Phase 1 \n\nMarket research and market compatibility testing: \n\nPerforming market survey for product. \nInvolves interviewing potential users and medical professionals. \nHelps understanding target audience and potential markets\n\nData collection and data preparation \n\nBuilding on existing MVP by increasing training data\nOptimizing data preparation methods to increase model performance\n\n\nPhase 2\n\nModel research \n\nResearching models compatible for current use case of classification\n\nModel training\n\nTraining the selected model with prepared data\nEvaluating model performance\nComparison of model performance with benchmarked models\n\nDeployment of model \n\nDeployment of model using Flask API\nAPI testing\n\nPhase 3\n\nLLM Integration\n\nResearching Large language models for treatment recommendation\nResearching strategies for LLM usage : API, Fine-tuning, Retrieval augmented generation(RAG)\nAnalysing LLM performance\n\nImproving User experience\n\nImproving UI to make

In [9]:
from google.colab import userdata
key = userdata.get('HF_TOKEN')

In [10]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
hf = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)


In [11]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.embeddings.spacy_embeddings import SpacyEmbeddings


text_splitter = SemanticChunker(HuggingFaceBgeEmbeddings())

In [12]:
docs = text_splitter.create_documents([data])



In [13]:
docs

[Document(page_content='Phase 1 \n\nMarket research and market compatibility testing: \n\nPerforming market survey for product.'),
 Document(page_content='Involves interviewing potential users and medical professionals. Helps understanding target audience and potential markets\n\nData collection and data preparation \n\nBuilding on existing MVP by increasing training data\nOptimizing data preparation methods to increase model performance\n\n\nPhase 2\n\nModel research \n\nResearching models compatible for current use case of classification\n\nModel training\n\nTraining the selected model with prepared data\nEvaluating model performance\nComparison of model performance with benchmarked models\n\nDeployment of model \n\nDeployment of model using Flask API\nAPI testing\n\nPhase 3\n\nLLM Integration\n\nResearching Large language models for treatment recommendation\nResearching strategies for LLM usage : API, Fine-tuning, Retrieval augmented generation(RAG)\nAnalysing LLM performance\n\nImp

In [14]:
len(docs)

2

#MILVUS Vector store

In [15]:
!pip install -U pymilvus

In [1]:
from pymilvus import MilvusClient

client = MilvusClient("test.db")

DEBUG:pymilvus.milvus_client.milvus_client:Created new connection using: d737c60b5dda46089416c14fbafd547e


In [2]:
if client.has_collection(collection_name="demo_collection"):
    client.drop_collection(collection_name="demo_collection")
client.create_collection(
    collection_name="demo_collection",
    dimension=768,  # vector dimension
    metric_type="L2",  # vector distance metric
    index_params={"index_type": "HNSW"},
)


DEBUG:pymilvus.milvus_client.milvus_client:Successfully created collection: demo_collection
DEBUG:pymilvus.milvus_client.milvus_client:Successfully created an index on collection: demo_collection


In [3]:
!pip install "pymilvus[model]"


In [16]:
from pymilvus import model

embedding_fn = model.DefaultEmbeddingFunction()

# Extract the text content from the Document objects
texts = [doc.page_content for doc in docs]

vectors = embedding_fn.encode_documents(texts)
print("Dim:", embedding_fn.dim, vectors[0].shape)  # Dim: 768 (768,)

data = [
    {"id": i, "vector": vectors[i], "text": texts[i], "subject": "history"}
    for i in range(len(vectors))
]

print("Data has", len(data), "entities, each with fields: ", data[0].keys())
print("Vector dim:", len(data[0]["vector"]))


Dim: 768 (768,)
Data has 2 entities, each with fields:  dict_keys(['id', 'vector', 'text', 'subject'])
Vector dim: 768


In [17]:
res = client.insert(collection_name="demo_collection", data=data)  # Use 'data' instead of 'docs'

print(res)




{'insert_count': 2, 'ids': [0, 1], 'cost': 0}
